# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

In [1]:
import os
from langchain.embeddings import QianfanEmbeddingsEndpoint
from langchain.chat_models import QianfanChatEndpoint

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

print(f"QIANFAN_AK={os.environ['QIANFAN_AK']}")
print(f"QIANFAN_SK={os.environ['QIANFAN_SK']}")

import warnings
warnings.filterwarnings("ignore")

QIANFAN_AK=HCCPsQy5p0Ex1rSEL6oorGQb
QIANFAN_SK=TCVRfPCfbtLr0eDPZRXcywMxcgaNuLDE


Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

~~~
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"
~~~

## Built-in LangChain tools

In [2]:
#!pip install -U wikipedia
#!pip install numexpr

In [16]:
# from langchain.agents.agent_toolkits import create_python_agent  # 报错，参考:https://stackoverflow.com/questions/77627546/unable-to-import-create-pandas-dataframe-agent-due-to-langchain-core-and-langcha
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
# from langchain.tools.python.tool import PythonREPLTool   # 报错，参考: https://github.com/QwenLM/Qwen/issues/873
from langchain_experimental.tools.python.tool  import PythonREPLTool
from langchain.python import PythonREPL
# from langchain.chat_models import ChatOpenAI

In [10]:
# llm = ChatOpenAI(temperature=0, model=llm_model)
llm = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = 0.01) 

In [11]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [12]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [13]:
agent("What is the 25% of 300?")

[INFO] [05-05 10:43:53] oauth.py:222 [t:31852]: trying to refresh access_token for ak `HCCPsQ***`




> Entering new AgentExecutor chain...


[INFO] [05-05 10:43:55] oauth.py:237 [t:31852]: sucessfully refresh access_token


Could not parse LLM output: 我们要计算300的25%是多少。
百分比是一个比例，它表示一个数是另一个数的多少部分。
在这个问题中，我们要找出300的25%是多少，这意味着我们要找出300的四分之一是多少，因为25%就是四分之一。

为了计算百分比，我们通常使用以下公式：
百分比值 = (部分值 / 全值) × 100%
但在这个问题中，我们已经知道百分比（25%）和全值（300），所以我们要求的是部分值。
因此，我们的公式变为：
部分值 = (百分比值 / 100%) × 全值
代入给定的值，我们得到：
部分值 = (25% / 100%) × 300

现在我们可以开始计算了。
计算结果为：部分值 = 75
所以，300的25%是：75。
Observation: Invalid or incomplete response
Thought:很抱歉，我之前的回答有误。这次我会直接给出正确的计算过程和答案。

为了计算300的25%，我们可以使用以下公式：
部分值 = (百分比值 / 100) × 全值
代入给定的值，我们得到：
部分值 = (25 / 100) × 300 = 75

所以，300的25%是75。

Final Answer: 300的25%是75。

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '300的25%是75。'}

## Wikipedia example

In [14]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question) 



> Entering new AgentExecutor chain...
Question: Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). What book did he write?

Thought: To answer this question, I need to search for information about Tom M. Mitchell's published works.

Action:
```json
{
  "action": "wikipedia",
  "action_input": "Tom M. Mitchell books"
}
```

Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Science

## Python Agent

In [17]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [18]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [19]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


To sort the customers by last name and then first name, I can use the `sorted()` function in Python with a custom key function. The key function will return a tuple consisting of the last name and first name, which will be used for sorting.

Action: Python_REPL
Action Input:


```python
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[0], x[1]))
print(sorted_customers)
```
Observation: [['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]

Thought:The code I executed sorted the customers by last name and then first name, but it seems like the sorting order is not correct. The list is not sorted alphabetically by last name. I need to debug my code to fix this issue.

Action: Python_REPL
Action Input:


```python
customers = [['Harrison', 'Chase'], ['

"The sorted list of customers by last name and then first name (case-insensitive) for the original input is:\n\n[['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]\n\nHowever, it's important to note that the relative order of customers with the same last name initial might differ from the original input due to the stable sorting behavior."

#### View detailed outputs of the chains

In [20]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:QianfanChatEndpoint] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf y

## Define your own tool

In [21]:
#!pip install DateTime

In [22]:
from langchain.agents import tool
from datetime import date

In [23]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [24]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

**Note**: 

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). 

If it does, please try running it again.

In [25]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")



> Entering new AgentExecutor chain...
Could not parse LLM output: 2024年5月5日，星期日，立夏，法定节假日（2024年5月1日～2024年5月5日，共5天）。农历是甲辰年 己巳月 己巳日 (三月廿七)。生肖是龙。

宜：馀事勿取.结网.塞穴。忌：诸事不宜。
Observation: Invalid or incomplete response
Thought:Action:
```json
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2024-05-05
Thought:Final Answer: 今天的日期是2024年5月5日。但请注意，我之前的回答中包含了农历、生肖以及宜忌等信息，这些信息可能对您来说并不相关或有用。如果您只需要知道公历日期，那么今天的日期就是2024年5月5日。

> Finished chain.


Reminder: Download your notebook to you local computer to save your work.